# Enunciado
Hola ingeniero 👋. Bienvenido a tu segundo desafio! Ya sabes que en [Vault-Tec Corporation](https://fallout.fandom.com/es/wiki/Vault-Tec_Corporation) tenemos sensores encargados de monitorear las condiciones actuales de nuestros refugios. La vez pasada detectamos que varios de ellos presentaban fallas recurrentes, impidiendo que pudieramos determinar las condiciones de nuestros bóvedas y darle la seguridad esperada a nuestros clientes en un mundo post apocaliptico 🙃. 

Con las respuestas que nos diste anteriormente, nuestro equipo de mantenimiento fisico de las instalaciones ha podido reemplazar los sensores malfuncionantes por unos nuevos. Ahora necesitamos que revises los logs de estos nuevos sensores y nos ayudes a determinar si estan funcionando corractemente. 

Recuerda que la vez pasada los ingenieros de Vault Tech Corp (ahora en mejor vida 💀) te dejaron los datos listos para su analisis, esta vez TU seras responsable de estructurar los datos y ejecutar el respectivo analisis.

Tu tarea, si decides aceptarla, es encontrar cuantas fallas ha habido y que sensores son los que presentan fallas, para que asi nuestro equipo pueda reemplazarlos. 

# Insumos
El equipo encargado del monitoreo de la calidad de los sensores, te ha enviado los [logs](https://keepcoding.io/blog/que-son-logs-y-para-que-sirven/) de dichos sensores. 

Algunos ejemplos de logs de estos sensores se ven de la siguiente forma:
```python
[
    {'sensor_id': 'oxy_guard','event_type': 'FAILURE_START','timestamp': '2025-07-31 07:01:24','duration_seconds': 577},
    {'sensor_id': 'radi_shield','event_type': 'FAILURE_START','timestamp': '2025-07-31 07:19:02','duration_seconds': 542}   
]
```

Por ahora este mismo equipo se ha encargado de agrupar los logs, para que solo exista una entrada de inicio y fin de falla por sensor por dia, de esta manera, para el sensor `thermo_core` solo podra haber un evento de inicio de falla y un evento de fin de falla por dia.

In [1]:
import json

with open(r"../data/logs.json") as file:  # leamos el archivo JSON de logs
    logs: list[dict] = json.load(file)  # cargamos a JSON el archivo

logs[0:5]  # mostremos los primeros 5 logs

[{'sensor_id': 'hidra_flow',
  'event_type': 'FAILURE_END',
  'timestamp': '2025-07-31 08:18:30',
  'duration_seconds': 271},
 {'sensor_id': 'thermo_core',
  'event_type': 'FAILURE_END',
  'timestamp': '2025-07-31 10:29:02',
  'duration_seconds': 542},
 {'sensor_id': 'hidra_flow',
  'event_type': 'FAILURE_END',
  'timestamp': '2025-07-31 10:30:07',
  'duration_seconds': 12},
 {'sensor_id': 'hidra_flow',
  'event_type': 'FAILURE_START',
  'timestamp': '2025-07-31 10:38:43',
  'duration_seconds': 365},
 {'sensor_id': 'thermo_core',
  'event_type': 'FAILURE_START',
  'timestamp': '2025-07-31 10:42:42',
  'duration_seconds': 290}]

# Tu turno!
Ahora es tu turno! Vault-Tec Corp necesita saber lo siguiente:

Para una fecha y sensor arbitrarios. Cual es el tiempo total, en segundos, en el cual el sensor se encontro en estado de fallo para la fecha asignada?
    - Si el sensor asignado no existe retorne `None`.
    - Si la fecha asignada no se encuentra dentro de las fechas existentes en los logs retorne `None`. 

Para poder lograrlo tu equipo te recomienda seguir los siguientes pasos:

1. **Verificar que el sensor dado sea valido**: Verifica que el sensor_id dado si sea valido. 

💡 Recuerda que solo existen 4 tipos de sensores: hidra_flow, thermo_core, radi_shield y oxy_guard.

In [2]:
def is_valid_sensor_id(sensor_id: str) -> bool:
    """
    Verifica si un sensor_id dado es válido dentro de los sensores conocidos.

    Esta función comprueba si el identificador del sensor proporcionado
    corresponde a uno de los sensores que el sistema reconoce como válidos.
    Es útil para evitar errores al procesar logs o datos de sensores que
    no existen o no están registrados.

    Args:
        sensor_id (str): Identificador del sensor a validar.

    Returns:
        bool: Devuelve True si el sensor_id es válido, False si no lo es.
    """
    VALID_SENSOR_IDS: list[str] = ["hidra_flow", "thermo_core", "radi_shield", "oxy_guard"]
    return sensor_id in VALID_SENSOR_IDS

2. **Obtener todos los logs para una fecha y un tipo de sensor**: Obten todos los logs para un fecha y un tipo de sensor arbitrarios (dados). 

💡 Puedes transformar la fecha de string a date por medio del modulo datetime, recuerda que las fechas de los logs se encuentran en formato timestamp, por lo cual deberas de hacer algo para poder matchear la fecha ingresada con la fecha de los logs. 

In [3]:
from datetime import date, datetime


def fetch_logs_on_date(sensor_id: str, date: date, logs: list[dict]) -> list[dict]:
    """
    Filtra y devuelve los logs de un sensor específico correspondientes a una fecha determinada.

    Esta función toma todos los registros disponibles y selecciona únicamente aquellos
    que coinciden con el `sensor_id` proporcionado y cuya fecha del timestamp corresponde
    al día especificado. Es útil para analizar fallas de sensores en días concretos
    sin importar la hora exacta del evento.

    Args:
        sensor_id (str): Identificador del sensor que se desea filtrar.
        date (date): Fecha específica que se desea consultar.
        logs (List[Dict]): Lista de diccionarios que contienen los logs.
            Cada log debe tener al menos las claves:
            - 'sensor_id': str
            - 'timestamp': str en formato "%Y-%m-%d %H:%M:%S"

    Returns:
        List[Dict]: Lista de logs filtrados que cumplen con el `sensor_id` y la fecha.
            Cada elemento es un diccionario con la información original del log.
    """
    logs_on_date: list[dict] = list(
        filter(
            lambda x: (
                (datetime.strptime(x["timestamp"], "%Y-%m-%d %H:%M:%S").date() == date)
                and x["sensor_id"] == sensor_id
            ),
            logs,
        )
    )
    return logs_on_date

3. **Juntemoslo todo**: Usemos las funcionalidades pasadas para verificar que el sensor dado si sea valido, y luego que si existan logs para la fecha y tipo de sensor dados. Finalmente, obtengamos el tiempo total de falla, en segundos, para ese sensor en la fecha dada. 

💡 Recuerda que para 1 fecha solo puede haber una entrada de inicio y fin de falla.

In [ ]:
def get_sensor_failure_duration(
    sensor_id: str, date: str, logs: list[dict]
) -> dict[str, float] | None:
    """
    Calcula el tiempo total de falla de un sensor en un día específico.

    Esta función toma los logs de un sensor y calcula la diferencia en segundos
    entre el inicio (`FAILURE_START`) y el fin (`FAILURE_END`) de la falla
    para la fecha indicada. Es útil para monitorear el tiempo de inactividad
    de sensores en un refugio.

    Args:
        sensor_id (str): Identificador del sensor a consultar.
        date (str): Fecha en formato "YYYY-MM-DD" para la cual se calcula la falla.
        logs (List[Dict]): Lista de diccionarios con los registros de los sensores.
            Cada log debe contener al menos las claves:
            - 'sensor_id' (str)
            - 'timestamp' (str) en formato "%Y-%m-%d %H:%M:%S"
            - 'event_type' (str) con valores "FAILURE_START" o "FAILURE_END"
            - 'duration_seconds' (float) indicando la duración del evento.

    Returns:
        Optional[Dict[str, float]]: Diccionario con la información del sensor:
            - 'sensor_id': id del sensor
            - 'date': fecha consultada (objeto date)
            - 'failure_total_time': tiempo total de falla en segundos
        Devuelve None si el `sensor_id` no es válido o no hay logs para esa fecha.
    """
    if not is_valid_sensor_id(sensor_id):
        print(f"⚠️ Sensor ID '{sensor_id}' no es válido.")
        return None

    searched_date = datetime.strptime(date, "%Y-%m-%d").date()
    logs_on_date: list[dict] = fetch_logs_on_date(sensor_id, searched_date, logs)

    if not logs_on_date:  # lista vacía → no existen logs para ese sensor o fecha
        print(f"No se encontraron logs para el sensor '{sensor_id}' en la fecha {searched_date}.")
        return None

    failure_start_time: float = next(
        (log["duration_seconds"] for log in logs_on_date if log["event_type"] == "FAILURE_START"), 0
    )
    failure_end_time: float = next(
        (log["duration_seconds"] for log in logs_on_date if log["event_type"] == "FAILURE_END"), 0
    )

    return {
        "sensor_id": sensor_id,
        "date": searched_date,
        "failure_total_time": failure_end_time - failure_start_time,
    }

In [5]:
get_sensor_failure_duration("hidra_flow", "2025-07-31", logs)

{'sensor_id': 'hidra_flow',
 'date': datetime.date(2025, 7, 31),
 'failure_total_time': -94}